In [9]:
import numpy as np 
import matplotlib.pyplot as plt 
import os 
import subprocess

In [12]:
""" 
    The next step is to fetch the problem parameters from problem.ini.
    We define a function which will do it.
"""

def read_parameters(filename):
    #We define a dictionary to read key pair value
    parameter= {}
    #We need now to open filename
    with open(filename) as file:
        #I need now to read each line
        for line in file:
            if "=" in line:
                key , value = line.strip().split("=")
                parameter[key.strip()] = np.float64(value.strip())

    return parameter["x_min"] , parameter["x_max"] , parameter["delta_t"], parameter["diffusion_coefficient"]


x_min , x_max , delta_t , diffusion_coefficient = read_parameters("./Setup/problem.ini")


In [18]:
#Once we get the parameter value we can then  computegenerate intial conditon
print(x_min, x_max, delta_t, diffusion_coefficient)
x= np.linspace(0,1,100)
U_0 = np.sin(np.pi*x)
#Since U_0 is now computed We write it to a init.dat file in bynary form
U_0.astype(np.float64).tofile("./Setup/init.dat")

0.0 1.0 0.00454545 1.0


In [21]:
#Once the init.dat file is formed we can now run the heat_equation.exe
result = subprocess.run(["./Bin/heat_solver.exe"])
if( result.returncode !=0):
    print("Execution fail")
    exit(1)
#result allows us to have the Output



In [22]:
#Once the output is ready we need to add them into a single array.
#First we sort ensure timestep consitency
#By taking step vlaue between _ and . dat.

output_files = sorted([f for f in os.listdir("./Output/")
                       if f.startswith("output_") and f.endswith(".dat")],
                       key= lambda name : int(name.split("_")[1].split(".")[0])
                       )

#We check whether the outpufile is valid
if not output_files:
    print( "outpufiles is empty")
    exit(1)

In [ ]:
#Now that we sorted the output temperature into a outpufiles
#We now fetch eaxch line , convert it back to double 
#and write it back to have an array of temperature
Heat = []
for file in output_files:
        #We build the complete path
        path = os.path.join("./Output",file)
        U = np.fromfile(path, dtype= np.float64)
        Heat.append(U)

heat= np.array(Heat)
print(heat.shape)